In [74]:
from attributes import *
import polars as pl
import altair as alt

# get data
df = pl.read_csv("data/2_research_ethic_recognition.csv")

In [75]:
def basic_analysis(name):
    count = df.group_by(name).len().sort(name)
    proportion = count.with_columns(
        pct = pl.col("len") / pl.col("len").sum() * 100
    )
    mean = df.select(mean = pl.col(name).mean())
    std = df.select(std = pl.col(name).std())
    return count, proportion, mean, std
    

In [76]:
def draw_bar_plot(df, x, y, title, MAP, rotate=True, legend=True):

    bar_x = alt.X(x, title=title, axis=None)
    bar_x = bar_x if MAP is None else bar_x.sort(MAP.values())

    txt_x = alt.X(x, sort=MAP.values()) if MAP is not None else alt.X(x)
    lgd_x = alt.Legend(orient='bottom', direction='horizontal')
    color = alt.Color(x, title=None, legend=lgd_x) if legend else alt.Color(x, title=None, legend=None)
    color = color.sort(MAP.values()) if MAP is not None else color

    bar = df.plot.bar().encode(
        x=bar_x,
        y=alt.Y(y, title=None),
        color=color,
    ).properties(
        title=title,
        width=400,
    )

    text = bar.mark_text(
        align="center",
        baseline="bottom",
    ).encode(
        x=txt_x,
        y=y,
        text=y,
        color=alt.value("black"),
    )
    return bar + text

In [77]:
from attributes import *
from scipy import stats
import pandas as pd

#
# Chi-square test
#
personal_info = pl.read_csv("data/6_personal_info.csv")
recognition_stats = df.join(personal_info, on="ID")

def chi2_test(target_name):
    chi_analysis = {}
    for key, value in MAP_PERSON.items():
        target = recognition_stats.pivot(target_name, index=value, values=target_name, aggregate_function='count', sort_columns=True).fill_null(0).drop(value)
        t, p, dof, _ = stats.chi2_contingency(target.to_numpy())
        chi_analysis[key] = (float(t), float(p))
    return chi_analysis

In [78]:
# 
# 가. 연구자의 연구윤리 인식과 실천의 중요성 인식 
# 
title = "연구윤리 인식 및 실천의 중요성 인식"

# 1) counting
print(chi2_test("Q1"))
basic_analysis("Q1")

# 
# plot
# 
# bar = draw_bar_plot(rer_value_counts, "Q1:N", "count:Q", title, MAP_IMPORTANCE)
# bar.show()
# bar.save("figure/2/가_연구윤리_인식_및_실천의_중요성_인식.svg")

{'성별': (7.99604002716651, 0.04609361732308934), '연령대': (11.07713313131112, 0.5223201945388567), '직위': (28.12793529746116, 0.02078179368954146), '최종학위': (22.40100699926988, 0.0010240390342926235), '연구경력': (6.081327647915995, 0.731751883507517), '연구분야': (16.570689832174658, 0.5527833189614135)}


C:\Users\hyeon\AppData\Local\Temp\ipykernel_8008\795108283.py:14: DeprecationWarning: `aggregate_function='count'` input for `pivot` is deprecated. Please use `aggregate_function='len'`.
  target = recognition_stats.pivot(target_name, index=value, values=target_name, aggregate_function='count', sort_columns=True).fill_null(0).drop(value)


(shape: (4, 2)
 ┌─────┬─────┐
 │ Q1  ┆ len │
 │ --- ┆ --- │
 │ i64 ┆ u32 │
 ╞═════╪═════╡
 │ 2   ┆ 1   │
 │ 3   ┆ 2   │
 │ 4   ┆ 52  │
 │ 5   ┆ 112 │
 └─────┴─────┘,
 shape: (4, 3)
 ┌─────┬─────┬───────────┐
 │ Q1  ┆ len ┆ pct       │
 │ --- ┆ --- ┆ ---       │
 │ i64 ┆ u32 ┆ f64       │
 ╞═════╪═════╪═══════════╡
 │ 2   ┆ 1   ┆ 0.598802  │
 │ 3   ┆ 2   ┆ 1.197605  │
 │ 4   ┆ 52  ┆ 31.137725 │
 │ 5   ┆ 112 ┆ 67.065868 │
 └─────┴─────┴───────────┘,
 shape: (1, 1)
 ┌──────────┐
 │ mean     │
 │ ---      │
 │ f64      │
 ╞══════════╡
 │ 4.646707 │
 └──────────┘,
 shape: (1, 1)
 ┌──────────┐
 │ std      │
 │ ---      │
 │ f64      │
 ╞══════════╡
 │ 0.538604 │
 └──────────┘)

In [79]:
# 
# 나. 연구자의 연구윤리 준수 수준 
# 
print(chi2_test("Q2"))
basic_analysis("Q2")

# visualization
# bar = draw_bar_plot(Compilance, "Q2:N", "count:Q", title, None, legend=False)
# bar.show()
# bar.save("figure/2/나_연구윤리_준수_수준.svg")

{'성별': (18.490127299415835, 0.009943879369940423), '연령대': (41.41457743334756, 0.04918092860434572), '직위': (41.83218091654873, 0.1983934789391329), '최종학위': (40.9719884424418, 0.0001796868729557265), '연구경력': (22.769002766151893, 0.3564173164238591), '연구분야': (59.19083094684393, 0.041089791314793025)}


C:\Users\hyeon\AppData\Local\Temp\ipykernel_8008\795108283.py:14: DeprecationWarning: `aggregate_function='count'` input for `pivot` is deprecated. Please use `aggregate_function='len'`.
  target = recognition_stats.pivot(target_name, index=value, values=target_name, aggregate_function='count', sort_columns=True).fill_null(0).drop(value)


(shape: (8, 2)
 ┌─────┬─────┐
 │ Q2  ┆ len │
 │ --- ┆ --- │
 │ i64 ┆ u32 │
 ╞═════╪═════╡
 │ 3   ┆ 1   │
 │ 4   ┆ 3   │
 │ 5   ┆ 1   │
 │ 6   ┆ 8   │
 │ 7   ┆ 9   │
 │ 8   ┆ 54  │
 │ 9   ┆ 54  │
 │ 10  ┆ 37  │
 └─────┴─────┘,
 shape: (8, 3)
 ┌─────┬─────┬───────────┐
 │ Q2  ┆ len ┆ pct       │
 │ --- ┆ --- ┆ ---       │
 │ i64 ┆ u32 ┆ f64       │
 ╞═════╪═════╪═══════════╡
 │ 3   ┆ 1   ┆ 0.598802  │
 │ 4   ┆ 3   ┆ 1.796407  │
 │ 5   ┆ 1   ┆ 0.598802  │
 │ 6   ┆ 8   ┆ 4.790419  │
 │ 7   ┆ 9   ┆ 5.389222  │
 │ 8   ┆ 54  ┆ 32.335329 │
 │ 9   ┆ 54  ┆ 32.335329 │
 │ 10  ┆ 37  ┆ 22.155689 │
 └─────┴─────┴───────────┘,
 shape: (1, 1)
 ┌──────────┐
 │ mean     │
 │ ---      │
 │ f64      │
 ╞══════════╡
 │ 8.497006 │
 └──────────┘,
 shape: (1, 1)
 ┌──────────┐
 │ std      │
 │ ---      │
 │ f64      │
 ╞══════════╡
 │ 1.307413 │
 └──────────┘)

In [80]:
# 
# 다. 연구자의 연구윤리 준수 영향 요인별
# 
print("연구윤리교육")
print(chi2_test("Q3_1"))
basic_analysis("Q3_1")

연구윤리교육


(shape: (5, 2)
 ┌──────┬─────┐
 │ Q3_1 ┆ len │
 │ ---  ┆ --- │
 │ i64  ┆ u32 │
 ╞══════╪═════╡
 │ 1    ┆ 2   │
 │ 2    ┆ 4   │
 │ 3    ┆ 22  │
 │ 4    ┆ 88  │
 │ 5    ┆ 51  │
 └──────┴─────┘,
 shape: (5, 3)
 ┌──────┬─────┬───────────┐
 │ Q3_1 ┆ len ┆ pct       │
 │ ---  ┆ --- ┆ ---       │
 │ i64  ┆ u32 ┆ f64       │
 ╞══════╪═════╪═══════════╡
 │ 1    ┆ 2   ┆ 1.197605  │
 │ 2    ┆ 4   ┆ 2.39521   │
 │ 3    ┆ 22  ┆ 13.173653 │
 │ 4    ┆ 88  ┆ 52.694611 │
 │ 5    ┆ 51  ┆ 30.538922 │
 └──────┴─────┴───────────┘,
 shape: (1, 1)
 ┌─────────┐
 │ mean    │
 │ ---     │
 │ f64     │
 ╞═════════╡
 │ 4.08982 │
 └─────────┘,
 shape: (1, 1)
 ┌──────────┐
 │ std      │
 │ ---      │
 │ f64      │
 ╞══════════╡
 │ 0.797786 │
 └──────────┘)

In [81]:
# 
# 다. 연구자의 연구윤리 준수 영향 요인별
# 
print("연구윤리 관련 보도")
print(chi2_test("Q3_2"))
basic_analysis("Q3_2")


연구윤리 관련 보도


(shape: (5, 2)
 ┌──────┬─────┐
 │ Q3_2 ┆ len │
 │ ---  ┆ --- │
 │ i64  ┆ u32 │
 ╞══════╪═════╡
 │ 1    ┆ 5   │
 │ 2    ┆ 5   │
 │ 3    ┆ 34  │
 │ 4    ┆ 86  │
 │ 5    ┆ 37  │
 └──────┴─────┘,
 shape: (5, 3)
 ┌──────┬─────┬───────────┐
 │ Q3_2 ┆ len ┆ pct       │
 │ ---  ┆ --- ┆ ---       │
 │ i64  ┆ u32 ┆ f64       │
 ╞══════╪═════╪═══════════╡
 │ 1    ┆ 5   ┆ 2.994012  │
 │ 2    ┆ 5   ┆ 2.994012  │
 │ 3    ┆ 34  ┆ 20.359281 │
 │ 4    ┆ 86  ┆ 51.497006 │
 │ 5    ┆ 37  ┆ 22.155689 │
 └──────┴─────┴───────────┘,
 shape: (1, 1)
 ┌──────────┐
 │ mean     │
 │ ---      │
 │ f64      │
 ╞══════════╡
 │ 3.868263 │
 └──────────┘,
 shape: (1, 1)
 ┌──────────┐
 │ std      │
 │ ---      │
 │ f64      │
 ╞══════════╡
 │ 0.895443 │
 └──────────┘)

In [82]:
# 
# 다. 연구자의 연구윤리 준수 영향 요인별
# 
print("연구자 간 대화 및 멘토링")
print(chi2_test("Q3_3"))
basic_analysis("Q3_3")

연구자 간 대화 및 멘토링


(shape: (4, 2)
 ┌──────┬─────┐
 │ Q3_3 ┆ len │
 │ ---  ┆ --- │
 │ i64  ┆ u32 │
 ╞══════╪═════╡
 │ 2    ┆ 2   │
 │ 3    ┆ 11  │
 │ 4    ┆ 76  │
 │ 5    ┆ 78  │
 └──────┴─────┘,
 shape: (4, 3)
 ┌──────┬─────┬───────────┐
 │ Q3_3 ┆ len ┆ pct       │
 │ ---  ┆ --- ┆ ---       │
 │ i64  ┆ u32 ┆ f64       │
 ╞══════╪═════╪═══════════╡
 │ 2    ┆ 2   ┆ 1.197605  │
 │ 3    ┆ 11  ┆ 6.586826  │
 │ 4    ┆ 76  ┆ 45.508982 │
 │ 5    ┆ 78  ┆ 46.706587 │
 └──────┴─────┴───────────┘,
 shape: (1, 1)
 ┌──────────┐
 │ mean     │
 │ ---      │
 │ f64      │
 ╞══════════╡
 │ 4.377246 │
 └──────────┘,
 shape: (1, 1)
 ┌──────────┐
 │ std      │
 │ ---      │
 │ f64      │
 ╞══════════╡
 │ 0.664203 │
 └──────────┘)

In [83]:
# 
# 다. 연구자의 연구윤리 준수 영향 요인별
# 
print("관련 규정 및 지침")
print(chi2_test("Q3_4"))
basic_analysis("Q3_4")

관련 규정 및 지침


(shape: (4, 2)
 ┌──────┬─────┐
 │ Q3_4 ┆ len │
 │ ---  ┆ --- │
 │ i64  ┆ u32 │
 ╞══════╪═════╡
 │ 2    ┆ 5   │
 │ 3    ┆ 14  │
 │ 4    ┆ 77  │
 │ 5    ┆ 71  │
 └──────┴─────┘,
 shape: (4, 3)
 ┌──────┬─────┬───────────┐
 │ Q3_4 ┆ len ┆ pct       │
 │ ---  ┆ --- ┆ ---       │
 │ i64  ┆ u32 ┆ f64       │
 ╞══════╪═════╪═══════════╡
 │ 2    ┆ 5   ┆ 2.994012  │
 │ 3    ┆ 14  ┆ 8.383234  │
 │ 4    ┆ 77  ┆ 46.107784 │
 │ 5    ┆ 71  ┆ 42.51497  │
 └──────┴─────┴───────────┘,
 shape: (1, 1)
 ┌──────────┐
 │ mean     │
 │ ---      │
 │ f64      │
 ╞══════════╡
 │ 4.281437 │
 └──────────┘,
 shape: (1, 1)
 ┌──────────┐
 │ std      │
 │ ---      │
 │ f64      │
 ╞══════════╡
 │ 0.743536 │
 └──────────┘)

In [84]:
# 
# 다. 연구자의 연구윤리 준수 영향 요인별
# 
print("부정행위 검증 및 제재")
print(chi2_test("Q3_5"))
basic_analysis("Q3_5")

부정행위 검증 및 제재


(shape: (4, 2)
 ┌──────┬─────┐
 │ Q3_5 ┆ len │
 │ ---  ┆ --- │
 │ i64  ┆ u32 │
 ╞══════╪═════╡
 │ 2    ┆ 1   │
 │ 3    ┆ 14  │
 │ 4    ┆ 93  │
 │ 5    ┆ 59  │
 └──────┴─────┘,
 shape: (4, 3)
 ┌──────┬─────┬───────────┐
 │ Q3_5 ┆ len ┆ pct       │
 │ ---  ┆ --- ┆ ---       │
 │ i64  ┆ u32 ┆ f64       │
 ╞══════╪═════╪═══════════╡
 │ 2    ┆ 1   ┆ 0.598802  │
 │ 3    ┆ 14  ┆ 8.383234  │
 │ 4    ┆ 93  ┆ 55.688623 │
 │ 5    ┆ 59  ┆ 35.329341 │
 └──────┴─────┴───────────┘,
 shape: (1, 1)
 ┌──────────┐
 │ mean     │
 │ ---      │
 │ f64      │
 ╞══════════╡
 │ 4.257485 │
 └──────────┘,
 shape: (1, 1)
 ┌──────────┐
 │ std      │
 │ ---      │
 │ f64      │
 ╞══════════╡
 │ 0.630204 │
 └──────────┘)

In [85]:
# 
# 다. 연구자의 연구윤리 준수 영향 요인별
# 
print("학술지 및 학술 컨퍼런스 자료")
print(chi2_test("Q3_6"))
basic_analysis("Q3_6")

학술지 및 학술 컨퍼런스 자료


(shape: (5, 2)
 ┌──────┬─────┐
 │ Q3_6 ┆ len │
 │ ---  ┆ --- │
 │ i64  ┆ u32 │
 ╞══════╪═════╡
 │ 1    ┆ 2   │
 │ 2    ┆ 4   │
 │ 3    ┆ 30  │
 │ 4    ┆ 92  │
 │ 5    ┆ 39  │
 └──────┴─────┘,
 shape: (5, 3)
 ┌──────┬─────┬───────────┐
 │ Q3_6 ┆ len ┆ pct       │
 │ ---  ┆ --- ┆ ---       │
 │ i64  ┆ u32 ┆ f64       │
 ╞══════╪═════╪═══════════╡
 │ 1    ┆ 2   ┆ 1.197605  │
 │ 2    ┆ 4   ┆ 2.39521   │
 │ 3    ┆ 30  ┆ 17.964072 │
 │ 4    ┆ 92  ┆ 55.08982  │
 │ 5    ┆ 39  ┆ 23.353293 │
 └──────┴─────┴───────────┘,
 shape: (1, 1)
 ┌─────────┐
 │ mean    │
 │ ---     │
 │ f64     │
 ╞═════════╡
 │ 3.97006 │
 └─────────┘,
 shape: (1, 1)
 ┌──────────┐
 │ std      │
 │ ---      │
 │ f64      │
 ╞══════════╡
 │ 0.787134 │
 └──────────┘)

In [93]:
# 
# 다. 연구자의 연구윤리 준수 영향 요인별
# 
print("기타")

count = df.group_by("Q4").len().sort("Q4")

케이스퍼센트_1 = count.with_columns(
    pct = pl.col("len") / pl.col("len").sum() * 100
) 

전체퍼센트_1 = count.with_columns(
    pct = pl.col("len") / (167-95) * 100
).drop_nulls()

count, 케이스퍼센트_1, 전체퍼센트_1


기타


(shape: (11, 2)
 ┌───────────────────────┬─────┐
 │ Q4                    ┆ len │
 │ ---                   ┆ --- │
 │ str                   ┆ u32 │
 ╞═══════════════════════╪═════╡
 │ null                  ┆ 95  │
 │ ETC (Leader)          ┆ 1   │
 │ ETC (prevention)      ┆ 1   │
 │ 개인 양심 및 윤리     ┆ 30  │
 │ 규정과 지침           ┆ 8   │
 │ …                     ┆ …   │
 │ 연구 환경 및 준수     ┆ 10  │
 │ 연구윤리 교육 및 훈련 ┆ 4   │
 │ 의사소통과 협력       ┆ 5   │
 │ 임금 및 노동 환경     ┆ 2   │
 │ 평가, 처벌 및 책임    ┆ 5   │
 └───────────────────────┴─────┘,
 shape: (11, 3)
 ┌───────────────────────┬─────┬───────────┐
 │ Q4                    ┆ len ┆ pct       │
 │ ---                   ┆ --- ┆ ---       │
 │ str                   ┆ u32 ┆ f64       │
 ╞═══════════════════════╪═════╪═══════════╡
 │ null                  ┆ 95  ┆ 56.886228 │
 │ ETC (Leader)          ┆ 1   ┆ 0.598802  │
 │ ETC (prevention)      ┆ 1   ┆ 0.598802  │
 │ 개인 양심 및 윤리     ┆ 30  ┆ 17.964072 │
 │ 규정과 지침           ┆ 8   ┆ 4.790419  │
 │ …             

In [87]:
# 
# 라. KIOST 연구윤리 확립을 위한 자체 노력의 정도
# 
basic_analysis("Q5")

# bar = draw_bar_plot(effort_value_counts, "Q5:N", "count:Q", "KIOST 연구윤리 확립을 위한 자체 노력의 정도", MAP_GOOD)
# bar.save("figure/2/라_KIOST_연구윤리_확립을_위한_자체_노력의_정도.svg")

(shape: (5, 2)
 ┌─────┬─────┐
 │ Q5  ┆ len │
 │ --- ┆ --- │
 │ i64 ┆ u32 │
 ╞═════╪═════╡
 │ 1   ┆ 2   │
 │ 2   ┆ 2   │
 │ 3   ┆ 57  │
 │ 4   ┆ 78  │
 │ 5   ┆ 28  │
 └─────┴─────┘,
 shape: (5, 3)
 ┌─────┬─────┬───────────┐
 │ Q5  ┆ len ┆ pct       │
 │ --- ┆ --- ┆ ---       │
 │ i64 ┆ u32 ┆ f64       │
 ╞═════╪═════╪═══════════╡
 │ 1   ┆ 2   ┆ 1.197605  │
 │ 2   ┆ 2   ┆ 1.197605  │
 │ 3   ┆ 57  ┆ 34.131737 │
 │ 4   ┆ 78  ┆ 46.706587 │
 │ 5   ┆ 28  ┆ 16.766467 │
 └─────┴─────┴───────────┘,
 shape: (1, 1)
 ┌──────────┐
 │ mean     │
 │ ---      │
 │ f64      │
 ╞══════════╡
 │ 3.766467 │
 └──────────┘,
 shape: (1, 1)
 ┌──────────┐
 │ std      │
 │ ---      │
 │ f64      │
 ╞══════════╡
 │ 0.783459 │
 └──────────┘)

In [88]:
from scipy import stats

#
# Chi-square test
#
personal_info = pl.read_csv("data/6_personal_info.csv")
recognition_stats = df.join(personal_info, on="ID")

def run_chi2_test(target_name):
    chi_analysis = {}
    for key, value in MAP_PERSON.items():
        target = recognition_stats.pivot(target_name, index=value, values=target_name, aggregate_function='count', sort_columns=True).fill_null(0).drop(value)
        chi_analysis[key] = stats.chi2_contingency(target)
    return chi_analysis

chi_recognition = run_chi2_test("Q1")
chi_compliance = run_chi2_test("Q2")
chi_compliance_influencing_factor_1 = run_chi2_test("Q3_1")
chi_compliance_influencing_factor_2 = run_chi2_test("Q3_2")
chi_compliance_influencing_factor_3 = run_chi2_test("Q3_3")
chi_compliance_influencing_factor_4 = run_chi2_test("Q3_4")
chi_compliance_influencing_factor_5 = run_chi2_test("Q3_5")
chi_compliance_influencing_factor_6 = run_chi2_test("Q3_6")
chi_effort = run_chi2_test("Q5")

C:\Users\hyeon\AppData\Local\Temp\ipykernel_8008\3786758127.py:12: DeprecationWarning: `aggregate_function='count'` input for `pivot` is deprecated. Please use `aggregate_function='len'`.
  target = recognition_stats.pivot(target_name, index=value, values=target_name, aggregate_function='count', sort_columns=True).fill_null(0).drop(value)
